In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from google.colab.patches import cv2_imshow
import dlib
import scipy.misc
import numpy as np
import os
import imageio
import cv2
import time

weights = '/content/drive/My Drive/ML project/MODELS/mmod_human_face_detector.dat'
# initializing cnn face detector model
cnn_face_detector = dlib.cnn_face_detection_model_v1(weights)
# Get Face Detector from dlib
# This allows us to detect faces in images
# face_detector = dlib.get_frontal_face_detector()
# Get Pose Predictor from dlib
# This allows us to detect landmark points in faces and understand the pose/angle of the face
shape_predictor = dlib.shape_predictor('/content/drive/My Drive/ML project/MODELS/shape_predictor_68_face_landmarks.dat')
# Get the face recognition model
# This is what gives us the face encodings (numbers that identify the face of a particular person)
face_recognition_model = dlib.face_recognition_model_v1('/content/drive/My Drive/ML project/MODELS/dlib_face_recognition_resnet_model_v1.dat')
# This is the tolerance for face comparisons
# The lower the number - the stricter the comparison
# To avoid false matches, use lower value
# To avoid false negatives (i.e. faces of the same person doesn't match), use higher value
# 0.5-0.6 works well
TOLERANCE = 0.50

def rect_to_bb(face):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = face.rect.left()
    y = face.rect.top()
    w = face.rect.right() - x
    h = face.rect.bottom() - y
    return (x, y, w, h)

# This function will take an image and return its face encodings using the neural network
def get_face_encodings(path_to_image):
    # Load image using scipy
    # print(path_to_image)
    image = cv2.imread(path_to_image)
    # Detect faces using the face detector
    # detected_faces = face_detector(image, 1)
    res = cnn_face_detector(image,1)
    # detected_faces.rect = detected_faces
    shapes_faces = []
    for r in res:
        face = r.rect
        # print(face)
        # Get pose/landmarks of those faces
        # Will be used as an input to the function that computes face encodings
        # This allows the neural network to be able to produce similar numbers for faces of the same people, regardless of camera angle and/or face positioning in the image
        shapes_faces.append(shape_predictor(image, face))
    # For every face detected, compute the face encodings
    return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

def get_vid_encodings(v_image,face):
    # print(face)
    # detected_faces = face_detector(v_image, 1)
    # Get pose/landmarks of those faces
    # Will be used as an input to the function that computes face encodings
    # This allows the neural network to be able to produce similar numbers for faces of the same people, regardless of camera angle and/or face positioning in the image
    shapes_faces = shape_predictor(v_image, face)
    # For every face detected, compute the face encodings
    # return [np.array(face_recognition_model.compute_face_descriptor(v_image, face_pose, 1)) for face_pose in shapes_faces]
    return [np.array(face_recognition_model.compute_face_descriptor(v_image, shapes_faces, 1))]

# This function takes a list of known faces
def compare_face_encodings(known_faces, face):
    # Finds the difference between each known face and the given face (that we are comparing)
    # Calculate norm for the differences with each known face
    # Return an array with True/Face values based on whether or not a known face matched with the given face
    print(type(face))
    print(type(known_faces))
    # print(known_faces)
    # A match occurs when the (norm) difference between a known face and the given face is less than or equal to the TOLERANCE value
    return (np.linalg.norm(known_faces - face, axis=1) <= TOLERANCE)

# This function returns the name of the person whose image matches with the given face (or 'Not Found')
# known_faces is a list of face encodings
# names is a list of the names of people (in the same order as the face encodings - to match the name with an encoding)
# face is the face we are looking for
def find_match(known_faces, names, face):
    # Call compare_face_encodings to get a list of True/False values indicating whether or not there's a match
    matches = compare_face_encodings(known_faces, face)
    # Return the name of the first match
    count = 0
    for match in matches:
        if match:
            return names[count]
        count += 1
    # Return not found if no match found
    return 'Not Found'

# Get path to all the known images
# Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('/content/drive/My Drive/ML project/known faces/'))
# Sort in alphabetical order
image_filenames = sorted(image_filenames)
# Get full paths to images
paths_to_images = ['/content/drive/My Drive/ML project/known faces/' + x for x in image_filenames]
# List of face encodings we have
face_encodings = []
# Loop over images to get the encoding one by one
start = time.time()

for path_to_image in paths_to_images:
    # Get face encodings from the image
    face_encodings_in_image = get_face_encodings(path_to_image)
    # Make sure there's exactly one face in the image
    if len(face_encodings_in_image) != 1:
        print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
        exit()
    # Append the face encoding found in that image to the list of face encodings we have
    face_encodings.append(face_encodings_in_image[0])

end = time.time()
print("Execution Time for Encoding: {}".format(end-start))

# Get path to all the test images
# Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
test_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('/content/drive/My Drive/ML project/test/'))
# Get full paths to test images
paths_to_test_images = ['/content/drive/My Drive/ML project/test/' + x for x in test_filenames]
# Get list of names of people by eliminating the .JPG OR .PNG extension from image filenames
names = [x[:-4] for x in image_filenames]


def draw_border(img, pt1, pt2, color, thickness, r, d):
    x1,y1 = pt1
    x2,y2 = pt2
 
    # Top left
    cv2.line(img, (x1 + r, y1), (x1 + r + d, y1), color, thickness)
    cv2.line(img, (x1, y1 + r), (x1, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x1 + r, y1 + r), (r, r), 180, 0, 90, color, thickness)
 
    # Top right
    cv2.line(img, (x2 - r, y1), (x2 - r - d, y1), color, thickness)
    cv2.line(img, (x2, y1 + r), (x2, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x2 - r, y1 + r), (r, r), 270, 0, 90, color, thickness)
 
    # Bottom left
    cv2.line(img, (x1 + r, y2), (x1 + r + d, y2), color, thickness)
    cv2.line(img, (x1, y2 - r), (x1, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x1 + r, y2 - r), (r, r), 90, 0, 90, color, thickness)
 
    # Bottom right
    cv2.line(img, (x2 - r, y2), (x2 - r - d, y2), color, thickness)
    cv2.line(img, (x2, y2 - r), (x2, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x2 - r, y2 - r), (r, r), 0, 0, 90, color, thickness)

# draw each face separately
def draw_faces_cnn(data):
    # load the image
    # data = pyplot.imread(filename)
    # data = imagefile
    # load image from file
    # create the detector, using default weights
    # detector = MTCNN()
    # detect faces in the image
    # result_list = detector.detect_faces(data)
    # result_list = face_detector(data, 1)
    result_list = cnn_face_detector(data, 1)
    # print(result_list)
    # display faces on the original image
    # plot each face as a subplot
    for i in range(len(result_list)):
        # get coordinates
        # x1, y1, width, height = result_list[i]['box']
        x1, y1, width, height = rect_to_bb(result_list[i])
        x2, y2 = x1 + width, y1 + height
        draw_border(data, (x1, y1), (x2, y2), (0, 255, 0),4, 15, 10)
        image = data[y1-40:y2+40, x1-40:x2+40]
        res = result_list[i].rect
        cv2_imshow(image)
        # flag = check_faces_cnn(image,i)
        # res = face_detector(image, 1)
        # print(flag)
        flag = True # if len(res)==1 else False
        if flag==True:
            print("Face Detected")
            # (h,w) = image.shape[:2]
            # center = (w/2,h/2)
            # M = cv2.getRotationMatrix2D(center,270,1.0)
            # rotated270 = cv2.warpAffine(image,M,(h,w))
            # face_encodings_in_image = get_vid_encodings(rotated270)
            face_encodings_in_image = get_vid_encodings(data,res)
            if len(face_encodings_in_image) != 1:
                print("Please change image: - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
                continue
            # Find match for the face encoding found in this test image
            match = find_match(face_encodings, names, face_encodings_in_image[0])
            # match = find_match(face_encodings, names, face_encodings_in_image[0])
            # Print the path of test image and the corresponding match
            # " + path_to_image + "
            print("This is",match)
        else:
            print("No Face")
            continue

new_image_filenames = filter(lambda x: x.endswith('.jpg') or x.endswith('.png'), os.listdir('/content/drive/My Drive/ML project/test/'))
# Sort in alphabetical order
new_image_filenames = sorted(new_image_filenames)
# Get full paths to images
#paths_to_new_images = ['/content/drive/My Drive/ML project/test/' + x for x in new_image_filenames]
paths_to_new_images = [
                       # 'h2.jpg'
                       '/content/drive/My Drive/ML project/known faces/Hritik Roshan.jpg',
                       '/content/drive/My Drive/ML project/known faces/Yashvi Raythatha.jpg',
                       '/content/drive/My Drive/ML project/test/bean.jpg',
                       #'img1.jpg'
                       ]
start = time.time()
for x in paths_to_new_images:
    img = cv2.imread(x)
    print(x)
    cv2_imshow(img)
    # img1 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # cv2_imshow(img1)
    # img1[...,2]=50
    # cv2_imshow(img1)
    # img2 = cv2.cvtColor(img1, cv2.COLOR_HSV2BGR)
    # cv2_imshow(img)
    # cv2_imshow(img2)
    draw_faces_cnn(img)
    # draw_faces_cnn(img1)
    # draw_faces_cnn(img2)
end = time.time()
print("Recognition time {}".format(end - start))
cv2.destroyAllWindows()
print('Done')